In [21]:
from sklearn.metrics import f1_score
import lazypredict
import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.impute import SimpleImputer
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
import lightgbm
import xgboost
import sklearn
import catboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import IsolationForest
#from pyod.models.ecod import ECOD
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import KNNImputer

In [22]:
df_X_last = pd.read_csv("data/X_train_features.csv", index_col=0)
df_X = pd.read_csv("data/X_train_features_new.csv", index_col=0)
df_Y = pd.read_csv("data/y_train.csv", index_col="id")

Remove highly correlated features

In [4]:
def rm_corr(X):
    corr_matrix = X.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find features with correlation greater than 0.9
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    print("Removed columns: ", len(to_drop))
    # Drop features 
    X.drop(to_drop, axis=1, inplace=True)
    # X_test.drop(to_drop, axis=1, inplace=True)
    
    return X

In [5]:
df_X = rm_corr(df_X)

Removed columns:  64


In [23]:
df_X.shape

(5117, 268)

Normalize

In [24]:
df_X.replace([np.inf, -np.inf], np.nan, inplace=True)
transformer = RobustScaler()
X = transformer.fit_transform(df_X)

Impute missing values

In [25]:
imputer = KNNImputer(n_neighbors=200)
#imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
X = imputer.fit_transform(X)

In [26]:
clf = RandomForestClassifier()
clf.fit(X, df_Y)
pred = clf.predict(X)

In [27]:
X.shape

(5117, 268)

Simple Outlier Removal

In [311]:
Y = df_Y.to_numpy()
for i in range(3):
    class_i = np.squeeze(Y==i)
    ids = np.where(class_i)[0]
    X_c = X[class_i]
    # preds = IsolationForest(random_state=0, contamination = 0.06).fit_predict(X_c)
    clf = ECOD()
    clf.fit(X_c)
    preds = clf.predict(X_c)
    outlier_ids = ids[preds == 1] # minus 1 for isolation forest
    Y = np.delete(Y, outlier_ids, axis=0)
    X = np.delete(X, outlier_ids, axis=0)
    print(outlier_ids.shape[0])

303
45
148


In [9]:
X.shape

(5117, 111)

In [53]:
X = SelectKBest(f_classif, k=170).fit_transform(X, df_Y)

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, df_Y,test_size=.5)
classifiers = [("LGBMClassifier", lightgbm.LGBMClassifier), ("XGBClassifier", xgboost.XGBClassifier), \
               ("RandomForestClassifier", sklearn.ensemble.RandomForestClassifier), \
                ("ExtraTreesClassifier", sklearn.ensemble.ExtraTreesClassifier)]
# catboost classifier takes very long but is similar in performance to these other classifiers
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None) # classifiers=classifiers
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [01:07<00:00,  2.33s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.83,0.73,None,0.82,11.15
BernoulliNB,0.66,0.70,None,0.68,0.10
LGBMClassifier,0.83,0.69,None,0.82,12.04
GaussianNB,0.71,0.69,None,0.71,0.10
LogisticRegression,0.79,0.69,None,0.79,0.26
LinearDiscriminantAnalysis,0.79,0.68,None,0.79,0.28
NearestCentroid,0.67,0.68,None,0.68,0.07
SGDClassifier,0.77,0.68,None,0.77,0.52
LinearSVC,0.79,0.67,None,0.79,4.02


In [63]:
Y = df_Y.to_numpy()
last = df_X_last.to_numpy()

In [66]:
skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
f1_last = []
f1_ours = []
for train_index, test_index in skf.split(X, Y):
    clf = RandomForestClassifier()
    
    clf.fit(X[train_index], Y[train_index])
    y_pred = clf.predict(X[test_index])
    f1_ours.append(f1_score(Y[test_index], y_pred, average='micro'))
    
    clf.fit(last[train_index], Y[train_index])
    y_pred = clf.predict(last[test_index])
    f1_last.append(f1_score(Y[test_index], y_pred, average='micro'))

In [72]:
# old feature version
print("Ours | Average: ", np.mean(f1_ours), "Std: ", np.std(f1_ours))
print("Theirs | Average: ", np.mean(f1_last), "Std: ", np.std(f1_last))

Ours | Average:  0.829394626710655 Std:  0.008750962139280984
Theirs | Average:  0.8458077498778105 Std:  0.01066281107735506


In [65]:
# old feature version
print("Ours | Average: ", np.mean(f1_ours), "Std: ", np.std(f1_ours))
print("Theirs | Average: ", np.mean(f1_last), "Std: ", np.std(f1_last))

Ours | Average:  0.840141396322092 Std:  0.006176959267631727
Theirs | Average:  0.8458077498778105 Std:  0.01066281107735506


In [67]:
# old feature version
print("Ours | Average: ", np.mean(f1_ours), "Std: ", np.std(f1_ours))
print("Theirs | Average: ", np.mean(f1_last), "Std: ", np.std(f1_last))

Ours | Average:  0.8280251481549366 Std:  0.0023249505986810692
Theirs | Average:  0.8362324734237537 Std:  0.004468336465268873
